# Hamiltonian Monte Carlo (HMC): Theory and Implementation from Scratch

## Introduction

Hamiltonian Monte Carlo (HMC), also known as Hybrid Monte Carlo, is an advanced Markov Chain Monte Carlo (MCMC) method that uses concepts from Hamiltonian dynamics to propose states that are distant from the current state while maintaining a high acceptance probability. This approach significantly reduces the random walk behavior that plagues simpler methods like Metropolis and Gibbs sampling, making HMC particularly effective for high-dimensional problems with complex geometries.

In this notebook, we will:
1. Explore the theoretical foundations of the Hamiltonian Monte Carlo algorithm
2. Implement the algorithm from scratch in Python
3. Apply it to sample from multivariate normal distributions
4. Extend it to a more complex problem: Bayesian logistic regression
5. Analyze the algorithm's performance and limitations

Let's begin by importing the necessary libraries.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import pandas as pd
from tqdm.notebook import tqdm
import autograd.numpy as anp
from autograd import grad

# Set random seed for reproducibility
np.random.seed(42)

# Set plot style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context('notebook')

## 1. Theoretical Background

### 1.1 The Hamiltonian Monte Carlo Algorithm

Hamiltonian Monte Carlo (HMC) uses concepts from physics to propose states that are distant from the current state but still have a high probability of acceptance. It introduces auxiliary momentum variables and uses Hamiltonian dynamics to guide the proposals.

For a target distribution $\pi(\mathbf{q}) \propto \exp(-U(\mathbf{q}))$, where $U(\mathbf{q})$ is the "potential energy" (negative log probability), the algorithm works as follows:

1. Introduce momentum variables $\mathbf{p}$ with distribution $\pi(\mathbf{p}) \propto \exp(-K(\mathbf{p}))$, typically $K(\mathbf{p}) = \frac{1}{2}\mathbf{p}^T\mathbf{M}^{-1}\mathbf{p}$ (multivariate normal with precision matrix $\mathbf{M}^{-1}$)

2. Define the Hamiltonian $H(\mathbf{q}, \mathbf{p}) = U(\mathbf{q}) + K(\mathbf{p})$

3. For each iteration $t$:
   a. Sample new momentum $\mathbf{p} \sim \mathcal{N}(\mathbf{0}, \mathbf{M})$
   b. Simulate Hamiltonian dynamics for $L$ steps with step size $\epsilon$:
      - Leapfrog integration:
        1. $\mathbf{p}' = \mathbf{p} - \frac{\epsilon}{2}\nabla U(\mathbf{q})$
        2. $\mathbf{q}' = \mathbf{q} + \epsilon\mathbf{M}^{-1}\mathbf{p}'$
        3. $\mathbf{p}' = \mathbf{p}' - \frac{\epsilon}{2}\nabla U(\mathbf{q}')$
   c. Calculate acceptance probability $\alpha = \min(1, \exp(H(\mathbf{q}, \mathbf{p}) - H(\mathbf{q}', \mathbf{p}')))$
   d. Accept or reject the proposal using Metropolis criterion

### 1.2 Key Properties

- **Gradient Information**: HMC uses the gradient of the log probability (potential energy) to guide the proposals, which helps it navigate the parameter space more efficiently.

- **Hamiltonian Dynamics**: The Hamiltonian dynamics simulation allows the algorithm to propose states that are distant from the current state but still have a high probability of acceptance.

- **Leapfrog Integration**: The leapfrog integrator is a symplectic integrator that preserves volume in phase space and is reversible, which is important for detailed balance.

- **Tuning Parameters**: HMC has two main tuning parameters: the step size $\epsilon$ and the number of leapfrog steps $L$. The step size affects the accuracy of the simulation, while the number of steps affects the distance of the proposals.

### 1.3 Practical Considerations

- **Gradient Computation**: HMC requires the gradient of the log probability, which can be computed analytically or using automatic differentiation.

- **Mass Matrix**: The mass matrix $\mathbf{M}$ can be adapted to the geometry of the target distribution. A common choice is the identity matrix, but using the inverse of the covariance matrix can improve efficiency.

- **Step Size and Number of Steps**: These parameters need to be tuned for optimal performance. The step size should be small enough for accurate simulation but large enough for efficient exploration. The number of steps should be large enough to propose distant states but small enough to keep the computational cost reasonable.

- **Adaptive HMC**: Extensions like the No-U-Turn Sampler (NUTS) automatically tune the number of leapfrog steps, while dual averaging can tune the step size.

Now, let's implement the Hamiltonian Monte Carlo algorithm from scratch.

## 2. Implementation from Scratch

We'll start by implementing the leapfrog integrator, which is the core component of the HMC algorithm.

In [ ]:
def leapfrog_integrator(q_current, p_current, grad_U, epsilon, L, M_inv=None):
    """
    Leapfrog integrator for Hamiltonian dynamics.
    
    Parameters:
    -----------
    q_current : array-like
        Current position (parameter values)
    p_current : array-like
        Current momentum
    grad_U : function
        Function that computes the gradient of the potential energy (negative log probability)
    epsilon : float
        Step size for the integrator
    L : int
        Number of leapfrog steps
    M_inv : array-like, optional
        Inverse mass matrix. If None, the identity matrix is used.
        
    Returns:
    --------
    q_proposed : array-like
        Proposed position after L leapfrog steps
    p_proposed : array-like
        Proposed momentum after L leapfrog steps
    """
    # Initialize
    q = np.copy(q_current)
    p = np.copy(p_current)
    
    # Set inverse mass matrix to identity if not provided
    if M_inv is None:
        M_inv = np.eye(len(q))
    
    # First half-step for momentum
    p = p - 0.5 * epsilon * grad_U(q)
    
    # Full leapfrog steps
    for i in range(L):
        # Full step for position
        q = q + epsilon * M_inv @ p
        
        # Full step for momentum, except at the end of the trajectory
        if i < L - 1:
            p = p - epsilon * grad_U(q)
        else:
            # Half-step for momentum at the end
            p = p - 0.5 * epsilon * grad_U(q)
    
    return q, p

Now, let's implement the full HMC algorithm.

In [ ]:
def hamiltonian_monte_carlo(U, grad_U, initial_position, n_samples, epsilon, L, M=None, burn_in=0, thin=1):
    """
    Hamiltonian Monte Carlo algorithm for sampling from a probability distribution.
    
    Parameters:
    -----------
    U : function
        Function that computes the potential energy (negative log probability)
    grad_U : function
        Function that computes the gradient of the potential energy
    initial_position : array-like
        Initial position (parameter values)
    n_samples : int
        Number of samples to generate
    epsilon : float
        Step size for the leapfrog integrator
    L : int
        Number of leapfrog steps
    M : array-like, optional
        Mass matrix. If None, the identity matrix is used.
    burn_in : int, optional
        Number of initial samples to discard
    thin : int, optional
        Thinning factor (keep every thin-th sample)
        
    Returns:
    --------
    samples : array
        Generated samples
    acceptance_rate : float
        Fraction of proposals that were accepted
    """
    # Convert initial position to numpy array if it isn't already
    q_current = np.array(initial_position, dtype=float)
    dim = q_current.shape[0]  # Dimensionality of the position
    
    # Set mass matrix to identity if not provided
    if M is None:
        M = np.eye(dim)
    
    # Compute inverse mass matrix
    M_inv = np.linalg.inv(M)
    
    # Total number of iterations needed
    n_iterations = burn_in + thin * n_samples
    
    # Initialize storage for samples and acceptance tracking
    samples = np.zeros((n_samples, dim))
    n_accepted = 0
    
    # Main sampling loop
    sample_idx = 0
    for i in tqdm(range(n_iterations), desc="Sampling"):
        # Resample momentum
        p_current = np.random.multivariate_normal(np.zeros(dim), M)
        
        # Compute Hamiltonian (energy) at current state
        current_U = U(q_current)
        current_K = 0.5 * p_current @ M_inv @ p_current
        current_H = current_U + current_K
        
        # Simulate Hamiltonian dynamics using leapfrog integrator
        q_proposed, p_proposed = leapfrog_integrator(q_current, p_current, grad_U, epsilon, L, M_inv)
        
        # Compute Hamiltonian (energy) at proposed state
        proposed_U = U(q_proposed)
        proposed_K = 0.5 * p_proposed @ M_inv @ p_proposed
        proposed_H = proposed_U + proposed_K
        
        # Metropolis acceptance step
        # Note: exp(current_H - proposed_H) = exp(-(proposed_H - current_H))
        # = exp(-(proposed_U - current_U + proposed_K - current_K))
        # = exp(-(proposed_U - current_U)) * exp(-(proposed_K - current_K))
        # = (proposed_density / current_density) * (proposed_momentum_density / current_momentum_density)
        delta_H = proposed_H - current_H
        if np.log(np.random.random()) < -delta_H:  # Accept with probability min(1, exp(-delta_H))
            q_current = q_proposed
            n_accepted += 1
        
        # Store the sample if past burn-in and due for storage based on thinning
        if i >= burn_in and (i - burn_in) % thin == 0:
            samples[sample_idx] = q_current
            sample_idx += 1
    
    # Compute acceptance rate
    acceptance_rate = n_accepted / n_iterations
    
    return samples, acceptance_rate

## 3. Example 1: Sampling from a Multivariate Normal Distribution

Let's apply our HMC sampler to a multivariate normal distribution. This is a good test case because:
1. We know the true distribution
2. We can easily calculate the density and its gradient
3. We can visualize the results in 2D
4. It demonstrates how HMC can efficiently navigate correlated distributions

We'll sample from a 2D normal distribution with mean $\mu = [0, 0]$ and covariance matrix $\Sigma = \begin{bmatrix} 1 & \rho \\ \rho & 1 \end{bmatrix}$, where $\rho$ is the correlation coefficient.

In [ ]:
# Define parameters for the target distribution
mean = np.array([0, 0])
rho = 0.95  # High correlation to demonstrate HMC's advantage
cov = np.array([[1, rho], [rho, 1]])
precision = np.linalg.inv(cov)  # Precision matrix (inverse covariance)

# Define potential energy (negative log probability) and its gradient
def U(q):
    """Potential energy (negative log probability) for multivariate normal."""
    delta = q - mean
    return 0.5 * delta @ precision @ delta

def grad_U(q):
    """Gradient of potential energy for multivariate normal."""
    delta = q - mean
    return precision @ delta

# Set sampling parameters
initial_position = np.array([2, 2])  # Start away from the mean
n_samples = 5000
epsilon = 0.1  # Step size
L = 10  # Number of leapfrog steps
burn_in = 1000
thin = 1

# Run the HMC sampler
samples, acceptance_rate = hamiltonian_monte_carlo(
    U, grad_U, initial_position, n_samples, epsilon, L, burn_in=burn_in, thin=thin
)

print(f"Acceptance rate: {acceptance_rate:.2f}")

Now, let's visualize the results to see how well our sampler approximates the target distribution.

In [ ]:
def plot_2d_samples(samples, true_mean=None, true_cov=None, title="Samples", show_trajectory=False, n_trajectory=100):
    """
    Plot 2D samples with marginal distributions and optionally compare to true distribution.
    
    Parameters:
    -----------
    samples : array-like
        2D samples to plot
    true_mean : array-like, optional
        True mean vector for comparison
    true_cov : array-like, optional
        True covariance matrix for comparison
    title : str, optional
        Plot title
    show_trajectory : bool, optional
        Whether to show the trajectory of the first n_trajectory samples
    n_trajectory : int, optional
        Number of samples to include in trajectory plot
    """
    # Create a figure with a grid for the joint and marginal plots
    fig = plt.figure(figsize=(10, 8))
    gs = fig.add_gridspec(2, 2, width_ratios=[3, 1], height_ratios=[1, 3],
                         wspace=0.05, hspace=0.05)
    
    # Joint distribution plot
    ax_joint = fig.add_subplot(gs[1, 0])
    
    # Plot the samples
    ax_joint.scatter(samples[:, 0], samples[:, 1], alpha=0.5, s=5)
    
    # If requested, show the trajectory of the first n_trajectory samples
    if show_trajectory and n_trajectory > 0:
        n = min(n_trajectory, len(samples))
        ax_joint.plot(samples[:n, 0], samples[:n, 1], 'r-', alpha=0.5, linewidth=0.5)
        ax_joint.scatter(samples[0, 0], samples[0, 1], color='red', s=30, label='Start')
    
    # If true distribution is provided, plot contours
    if true_mean is not None and true_cov is not None:
        # Create a grid of points
        x = np.linspace(-3, 3, 100)
        y = np.linspace(-3, 3, 100)
        X, Y = np.meshgrid(x, y)
        pos = np.dstack((X, Y))
        
        # Compute PDF values on the grid
        rv = stats.multivariate_normal(true_mean, true_cov)
        Z = rv.pdf(pos)
        
        # Plot contours
        levels = np.linspace(0, Z.max(), 10)[1:]
        ax_joint.contour(X, Y, Z, levels=levels, colors='r', alpha=0.7)
    
    # Set labels and limits
    ax_joint.set_xlabel('$q_1$')
    ax_joint.set_ylabel('$q_2$')
    ax_joint.set_xlim(-3, 3)
    ax_joint.set_ylim(-3, 3)
    
    # Marginal distribution for q1
    ax_marg_x = fig.add_subplot(gs[0, 0], sharex=ax_joint)
    sns.kdeplot(samples[:, 0], ax=ax_marg_x, fill=True)
    if true_mean is not None and true_cov is not None:
        x = np.linspace(-3, 3, 1000)
        ax_marg_x.plot(x, stats.norm.pdf(x, true_mean[0], np.sqrt(true_cov[0, 0])), 'r')
    ax_marg_x.set_yticks([])
    ax_marg_x.set_title(title)
    
    # Marginal distribution for q2
    ax_marg_y = fig.add_subplot(gs[1, 1], sharey=ax_joint)
    sns.kdeplot(y=samples[:, 1], ax=ax_marg_y, fill=True)
    if true_mean is not None and true_cov is not None:
        y = np.linspace(-3, 3, 1000)
        ax_marg_y.plot(stats.norm.pdf(y, true_mean[1], np.sqrt(true_cov[1, 1])), y, 'r')
    ax_marg_y.set_xticks([])
    
    # Turn off tick labels on the marginal plots
    plt.setp(ax_marg_x.get_xticklabels(), visible=False)
    plt.setp(ax_marg_y.get_yticklabels(), visible=False)
    
    if show_trajectory:
        ax_joint.legend()
    
    plt.show()

In [ ]:
# Plot the samples
plot_2d_samples(samples, mean, cov, title=f"HMC Samples (ρ={rho}, acceptance rate={acceptance_rate:.2f})")

In [ ]:
# Plot with trajectory to show how HMC explores the space
plot_2d_samples(samples, mean, cov, title=f"HMC Sampling Trajectory (ρ={rho})", show_trajectory=True, n_trajectory=100)

Let's also look at the trace plots and autocorrelation to assess mixing and convergence.

In [ ]:
def plot_diagnostics(samples, parameter_names=None):
    """
    Plot trace plots and autocorrelation for MCMC samples.
    
    Parameters:
    -----------
    samples : array-like
        MCMC samples
    parameter_names : list, optional
        Names of the parameters
    """
    n_samples, dim = samples.shape
    
    if parameter_names is None:
        parameter_names = [f"$q_{i+1}$" for i in range(dim)]
    
    # Create figure
    fig, axes = plt.subplots(dim, 2, figsize=(12, 3*dim))
    
    # Plot trace and autocorrelation for each parameter
    for i in range(dim):
        # Trace plot
        axes[i, 0].plot(samples[:, i])
        axes[i, 0].set_title(f"Trace plot for {parameter_names[i]}")
        axes[i, 0].set_xlabel("Iteration")
        axes[i, 0].set_ylabel(parameter_names[i])
        
        # Autocorrelation plot
        lags = min(50, n_samples // 5)
        acf = np.zeros(lags)
        for lag in range(lags):
            acf[lag] = np.corrcoef(samples[lag:, i], samples[:-lag if lag > 0 else None, i])[0, 1]
        
        axes[i, 1].bar(range(lags), acf)
        axes[i, 1].set_title(f"Autocorrelation for {parameter_names[i]}")
        axes[i, 1].set_xlabel("Lag")
        axes[i, 1].set_ylabel("Autocorrelation")
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Plot diagnostics
plot_diagnostics(samples)

### 3.1 Effect of Step Size and Number of Leapfrog Steps

The performance of HMC depends on two key parameters: the step size $\epsilon$ and the number of leapfrog steps $L$. Let's experiment with different values to see how they affect the acceptance rate and mixing.

In [ ]:
# Function to calculate effective sample size (ESS)
def calculate_ess(chain):
    """
    Calculate effective sample size using autocorrelation.
    
    Parameters:
    -----------
    chain : array-like
        MCMC chain
        
    Returns:
    --------
    ess : float
        Effective sample size
    """
    n = len(chain)
    lags = min(50, n // 5)
    acf = np.zeros(lags)
    for lag in range(lags):
        acf[lag] = np.corrcoef(chain[lag:], chain[:-lag if lag > 0 else None])[0, 1]
    # Truncate at first negative autocorrelation
    neg_idx = np.where(acf < 0)[0]
    if len(neg_idx) > 0:
        acf = acf[:neg_idx[0]]
    # Calculate ESS
    ess = n / (1 + 2 * np.sum(acf[1:]))
    return ess

In [ ]:
# Try different step sizes
epsilon_values = [0.01, 0.05, 0.1, 0.2, 0.5]
L = 10  # Fixed number of leapfrog steps
step_size_results = []

for epsilon in epsilon_values:
    samples, acceptance_rate = hamiltonian_monte_carlo(
        U, grad_U, initial_position, n_samples=2000, epsilon=epsilon, L=L, burn_in=500, thin=1
    )
    
    # Calculate effective sample size
    ess_q1 = calculate_ess(samples[:, 0])
    ess_q2 = calculate_ess(samples[:, 1])
    ess_avg = (ess_q1 + ess_q2) / 2
    
    step_size_results.append({
        'epsilon': epsilon,
        'acceptance_rate': acceptance_rate,
        'ess_q1': ess_q1,
        'ess_q2': ess_q2,
        'ess_avg': ess_avg,
        'efficiency': ess_avg / 2000  # ESS per iteration
    })
    
    # Plot samples
    plot_2d_samples(samples, mean, cov, 
                   title=f"Step size ε={epsilon}, Acceptance rate={acceptance_rate:.2f}, ESS={ess_avg:.1f}")

In [ ]:
# Summarize step size results
step_size_df = pd.DataFrame(step_size_results)
step_size_df

In [ ]:
# Plot acceptance rate and efficiency vs step size
fig, ax1 = plt.subplots(figsize=(10, 6))

ax1.set_xlabel('Step Size (ε)')
ax1.set_ylabel('Acceptance Rate', color='tab:blue')
ax1.plot(step_size_df['epsilon'], step_size_df['acceptance_rate'], 'o-', color='tab:blue')
ax1.tick_params(axis='y', labelcolor='tab:blue')

ax2 = ax1.twinx()
ax2.set_ylabel('Sampling Efficiency (ESS/N)', color='tab:red')
ax2.plot(step_size_df['epsilon'], step_size_df['efficiency'], 'o-', color='tab:red')
ax2.tick_params(axis='y', labelcolor='tab:red')

plt.title('Effect of Step Size on HMC Sampling')
fig.tight_layout()
plt.show()

In [ ]:
# Try different numbers of leapfrog steps
L_values = [1, 5, 10, 20, 50]
epsilon = 0.1  # Fixed step size
leapfrog_steps_results = []

for L in L_values:
    samples, acceptance_rate = hamiltonian_monte_carlo(
        U, grad_U, initial_position, n_samples=2000, epsilon=epsilon, L=L, burn_in=500, thin=1
    )
    
    # Calculate effective sample size
    ess_q1 = calculate_ess(samples[:, 0])
    ess_q2 = calculate_ess(samples[:, 1])
    ess_avg = (ess_q1 + ess_q2) / 2
    
    leapfrog_steps_results.append({
        'L': L,
        'acceptance_rate': acceptance_rate,
        'ess_q1': ess_q1,
        'ess_q2': ess_q2,
        'ess_avg': ess_avg,
        'efficiency': ess_avg / 2000,  # ESS per iteration
        'efficiency_per_grad': ess_avg / (2000 * L)  # ESS per gradient evaluation
    })
    
    # Plot samples
    plot_2d_samples(samples, mean, cov, 
                   title=f"Leapfrog steps L={L}, Acceptance rate={acceptance_rate:.2f}, ESS={ess_avg:.1f}")

In [ ]:
# Summarize leapfrog steps results
leapfrog_steps_df = pd.DataFrame(leapfrog_steps_results)
leapfrog_steps_df

In [ ]:
# Plot acceptance rate and efficiency vs number of leapfrog steps
fig, ax1 = plt.subplots(figsize=(10, 6))

ax1.set_xlabel('Number of Leapfrog Steps (L)')
ax1.set_ylabel('Acceptance Rate', color='tab:blue')
ax1.plot(leapfrog_steps_df['L'], leapfrog_steps_df['acceptance_rate'], 'o-', color='tab:blue')
ax1.tick_params(axis='y', labelcolor='tab:blue')

ax2 = ax1.twinx()
ax2.set_ylabel('Sampling Efficiency (ESS/N)', color='tab:red')
ax2.plot(leapfrog_steps_df['L'], leapfrog_steps_df['efficiency'], 'o-', color='tab:red')
ax2.tick_params(axis='y', labelcolor='tab:red')

plt.title('Effect of Number of Leapfrog Steps on HMC Sampling')
fig.tight_layout()
plt.show()

In [ ]:
# Plot efficiency per gradient evaluation vs number of leapfrog steps
plt.figure(figsize=(10, 6))
plt.plot(leapfrog_steps_df['L'], leapfrog_steps_df['efficiency_per_grad'], 'o-')
plt.xlabel('Number of Leapfrog Steps (L)')
plt.ylabel('Efficiency per Gradient Evaluation (ESS/N/L)')
plt.title('Computational Efficiency vs Number of Leapfrog Steps')
plt.grid(True)
plt.show()

### 3.2 Comparison with Metropolis and Gibbs Sampling

Let's compare the performance of HMC with the Metropolis and Gibbs sampling algorithms for the same multivariate normal distribution with high correlation.

In [ ]:
# Metropolis algorithm
def metropolis_sampler(log_prob_func, initial_state, n_samples, proposal_std, burn_in=0, thin=1):
    """
    Metropolis algorithm for sampling from a probability distribution.
    
    Parameters:
    -----------
    log_prob_func : function
        Function that computes the log probability of a state
    initial_state : array-like
        Initial state of the Markov chain
    n_samples : int
        Number of samples to generate
    proposal_std : float or array-like
        Standard deviation of the Gaussian proposal distribution
    burn_in : int, optional
        Number of initial samples to discard
    thin : int, optional
        Thinning factor (keep every thin-th sample)
        
    Returns:
    --------
    samples : array
        Generated samples
    acceptance_rate : float
        Fraction of proposals that were accepted
    """
    # Convert initial state to numpy array if it isn't already
    current_state = np.array(initial_state, dtype=float)
    dim = current_state.shape[0]  # Dimensionality of the state
    
    # Total number of iterations needed
    n_iterations = burn_in + thin * n_samples
    
    # Initialize storage for samples and acceptance tracking
    samples = np.zeros((n_samples, dim))
    current_log_prob = log_prob_func(current_state)
    n_accepted = 0
    
    # Main sampling loop
    sample_idx = 0
    for i in tqdm(range(n_iterations), desc="Sampling"):
        # Propose a new state
        proposal = current_state + np.random.normal(0, proposal_std, size=dim)
        
        # Compute log probability of the proposed state
        proposal_log_prob = log_prob_func(proposal)
        
        # Compute log acceptance ratio
        log_acceptance_ratio = proposal_log_prob - current_log_prob
        
        # Accept or reject the proposal
        if np.log(np.random.random()) < log_acceptance_ratio:
            current_state = proposal
            current_log_prob = proposal_log_prob
            n_accepted += 1
        
        # Store the sample if past burn-in and due for storage based on thinning
        if i >= burn_in and (i - burn_in) % thin == 0:
            samples[sample_idx] = current_state
            sample_idx += 1
    
    # Compute acceptance rate
    acceptance_rate = n_accepted / n_iterations
    
    return samples, acceptance_rate

In [ ]:
# Gibbs sampler for bivariate normal
def gibbs_sampler_bivariate_normal(mean, cov, initial_state, n_samples, burn_in=0, thin=1):
    """
    Gibbs sampling for a bivariate normal distribution.
    
    Parameters:
    -----------
    mean : array-like
        Mean vector [mu_1, mu_2]
    cov : array-like
        Covariance matrix [[sigma_1^2, rho*sigma_1*sigma_2], [rho*sigma_1*sigma_2, sigma_2^2]]
    initial_state : array-like
        Initial state of the Markov chain
    n_samples : int
        Number of samples to generate
    burn_in : int, optional
        Number of initial samples to discard
    thin : int, optional
        Thinning factor (keep every thin-th sample)
        
    Returns:
    --------
    samples : array
        Generated samples
    """
    # Extract parameters
    mu_1, mu_2 = mean
    sigma_1_sq, rho_sigma_1_sigma_2, _, sigma_2_sq = cov.flatten()
    
    sigma_1 = np.sqrt(sigma_1_sq)
    sigma_2 = np.sqrt(sigma_2_sq)
    rho = rho_sigma_1_sigma_2 / (sigma_1 * sigma_2)
    
    # Conditional standard deviations
    cond_std_1 = sigma_1 * np.sqrt(1 - rho**2)
    cond_std_2 = sigma_2 * np.sqrt(1 - rho**2)
    
    # Convert initial state to numpy array if it isn't already
    current_state = np.array(initial_state, dtype=float)
    
    # Total number of iterations needed
    n_iterations = burn_in + thin * n_samples
    
    # Initialize storage for samples
    samples = np.zeros((n_samples, 2))
    
    # Main sampling loop
    sample_idx = 0
    for i in tqdm(range(n_iterations), desc="Sampling"):
        # Sample x1 given x2
        x2 = current_state[1]
        cond_mean_1 = mu_1 + rho * (sigma_1 / sigma_2) * (x2 - mu_2)
        current_state[0] = np.random.normal(cond_mean_1, cond_std_1)
        
        # Sample x2 given x1
        x1 = current_state[0]
        cond_mean_2 = mu_2 + rho * (sigma_2 / sigma_1) * (x1 - mu_1)
        current_state[1] = np.random.normal(cond_mean_2, cond_std_2)
        
        # Store the sample if past burn-in and due for storage based on thinning
        if i >= burn_in and (i - burn_in) % thin == 0:
            samples[sample_idx] = current_state
            sample_idx += 1
    
    return samples

In [ ]:
# Define log probability function for Metropolis
def log_prob(q):
    """Log probability for multivariate normal."""
    return -U(q)  # Negative potential energy

# Run Metropolis sampler
metropolis_samples, metropolis_acceptance_rate = metropolis_sampler(
    log_prob, initial_position, n_samples=2000, proposal_std=0.3, burn_in=500, thin=1
)

# Run Gibbs sampler
gibbs_samples = gibbs_sampler_bivariate_normal(
    mean, cov, initial_position, n_samples=2000, burn_in=500, thin=1
)

# Run HMC sampler with optimal parameters
hmc_samples, hmc_acceptance_rate = hamiltonian_monte_carlo(
    U, grad_U, initial_position, n_samples=2000, epsilon=0.1, L=10, burn_in=500, thin=1
)

In [ ]:
# Calculate effective sample size for each method
metropolis_ess_q1 = calculate_ess(metropolis_samples[:, 0])
metropolis_ess_q2 = calculate_ess(metropolis_samples[:, 1])
metropolis_ess_avg = (metropolis_ess_q1 + metropolis_ess_q2) / 2

gibbs_ess_q1 = calculate_ess(gibbs_samples[:, 0])
gibbs_ess_q2 = calculate_ess(gibbs_samples[:, 1])
gibbs_ess_avg = (gibbs_ess_q1 + gibbs_ess_q2) / 2

hmc_ess_q1 = calculate_ess(hmc_samples[:, 0])
hmc_ess_q2 = calculate_ess(hmc_samples[:, 1])
hmc_ess_avg = (hmc_ess_q1 + hmc_ess_q2) / 2

# Create comparison DataFrame
comparison_df = pd.DataFrame({
    'Method': ['Metropolis', 'Gibbs', 'HMC'],
    'ESS q1': [metropolis_ess_q1, gibbs_ess_q1, hmc_ess_q1],
    'ESS q2': [metropolis_ess_q2, gibbs_ess_q2, hmc_ess_q2],
    'ESS Avg': [metropolis_ess_avg, gibbs_ess_avg, hmc_ess_avg],
    'Efficiency (ESS/N)': [metropolis_ess_avg/2000, gibbs_ess_avg/2000, hmc_ess_avg/2000],
    'Acceptance Rate': [metropolis_acceptance_rate, 1.0, hmc_acceptance_rate]
})

comparison_df

In [ ]:
# Plot comparison of sampling methods
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Plot Metropolis samples
axes[0].scatter(metropolis_samples[:, 0], metropolis_samples[:, 1], alpha=0.5, s=5)
axes[0].set_title(f"Metropolis (ρ={rho}, ESS={metropolis_ess_avg:.1f})")
axes[0].set_xlabel('$q_1$')
axes[0].set_ylabel('$q_2$')
axes[0].set_xlim(-3, 3)
axes[0].set_ylim(-3, 3)

# Plot Gibbs samples
axes[1].scatter(gibbs_samples[:, 0], gibbs_samples[:, 1], alpha=0.5, s=5)
axes[1].set_title(f"Gibbs (ρ={rho}, ESS={gibbs_ess_avg:.1f})")
axes[1].set_xlabel('$q_1$')
axes[1].set_ylabel('$q_2$')
axes[1].set_xlim(-3, 3)
axes[1].set_ylim(-3, 3)

# Plot HMC samples
axes[2].scatter(hmc_samples[:, 0], hmc_samples[:, 1], alpha=0.5, s=5)
axes[2].set_title(f"HMC (ρ={rho}, ESS={hmc_ess_avg:.1f})")
axes[2].set_xlabel('$q_1$')
axes[2].set_ylabel('$q_2$')
axes[2].set_xlim(-3, 3)
axes[2].set_ylim(-3, 3)

# Plot contours of the true distribution on all subplots
x = np.linspace(-3, 3, 100)
y = np.linspace(-3, 3, 100)
X, Y = np.meshgrid(x, y)
pos = np.dstack((X, Y))
rv = stats.multivariate_normal(mean, cov)
Z = rv.pdf(pos)
levels = np.linspace(0, Z.max(), 10)[1:]

for ax in axes:
    ax.contour(X, Y, Z, levels=levels, colors='r', alpha=0.7)

plt.tight_layout()
plt.show()

In [ ]:
# Plot efficiency comparison
plt.figure(figsize=(10, 6))
plt.bar(comparison_df['Method'], comparison_df['Efficiency (ESS/N)'])
plt.xlabel('Method')
plt.ylabel('Sampling Efficiency (ESS/N)')
plt.title(f'Sampling Efficiency Comparison (ρ={rho})')
plt.grid(True, axis='y')
plt.show()

## 4. Example 2: Bayesian Logistic Regression

Now, let's apply HMC to a more complex problem: Bayesian logistic regression. This is a good test case because:
1. The posterior is not analytically tractable
2. The gradient can be computed
3. The model has moderate dimensionality
4. It shows HMC's advantage in higher dimensions

### 4.1 Generate Synthetic Data

In [ ]:
# Set true parameter values
true_beta = np.array([0.5, 2.0, -1.0])  # Intercept and coefficients
n_features = len(true_beta) - 1

# Generate synthetic data
n_data = 200
X = np.random.normal(0, 1, size=(n_data, n_features))
X_with_intercept = np.column_stack([np.ones(n_data), X])
logits = X_with_intercept @ true_beta
p = 1 / (1 + np.exp(-logits))
y = np.random.binomial(1, p)

# Plot the data
plt.figure(figsize=(10, 6))
plt.scatter(X[:, 0], X[:, 1], c=y, cmap='coolwarm', alpha=0.7)
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Synthetic Data for Bayesian Logistic Regression')
plt.colorbar(label='Class')
plt.grid(True)
plt.show()

### 4.2 Define the Bayesian Model

In Bayesian logistic regression, we model:

$$p(y_i = 1 | \mathbf{x}_i, \boldsymbol{\beta}) = \frac{1}{1 + \exp(-\mathbf{x}_i^T \boldsymbol{\beta})}$$

We need to define a prior for the parameters $\boldsymbol{\beta}$. Let's use a multivariate normal prior:

$$\boldsymbol{\beta} \sim \mathcal{N}(\mathbf{0}, \sigma^2 \mathbf{I})$$

Then, we'll sample from the posterior distribution:

$$p(\boldsymbol{\beta} | \mathbf{X}, \mathbf{y}) \propto p(\mathbf{y} | \mathbf{X}, \boldsymbol{\beta}) \cdot p(\boldsymbol{\beta})$$

Let's define the potential energy (negative log posterior) and its gradient using autograd for automatic differentiation.

In [ ]:
# Convert data to autograd numpy arrays
X_with_intercept_anp = anp.array(X_with_intercept)
y_anp = anp.array(y)

# Define potential energy (negative log posterior) using autograd
def U_logistic(beta, X=X_with_intercept_anp, y=y_anp, sigma=10.0):
    """Potential energy (negative log posterior) for Bayesian logistic regression."""
    # Prior: beta ~ N(0, sigma^2 * I)
    prior = anp.sum(beta**2) / (2 * sigma**2)
    
    # Likelihood: y ~ Bernoulli(sigmoid(X @ beta))
    logits = X @ beta
    log_likelihood = anp.sum(y * logits - anp.log(1 + anp.exp(logits)))
    
    # Negative log posterior
    return prior - log_likelihood

# Compute gradient using autograd
grad_U_logistic = grad(U_logistic)

### 4.3 Sample from the Posterior

In [ ]:
# Set sampling parameters
initial_position = np.zeros(len(true_beta))  # Start at zero
n_samples = 5000
epsilon = 0.01  # Step size
L = 20  # Number of leapfrog steps
burn_in = 1000
thin = 2

# Run the HMC sampler
posterior_samples, acceptance_rate = hamiltonian_monte_carlo(
    U_logistic, grad_U_logistic, initial_position, n_samples, epsilon, L, burn_in=burn_in, thin=thin
)

print(f"Acceptance rate: {acceptance_rate:.2f}")

### 4.4 Analyze the Results

In [ ]:
# Plot diagnostics
plot_diagnostics(posterior_samples, parameter_names=['Intercept', 'Coefficient 1', 'Coefficient 2'])

In [ ]:
# Plot posterior distributions
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for i in range(3):
    sns.histplot(posterior_samples[:, i], kde=True, ax=axes[i])
    axes[i].axvline(true_beta[i], color='r', linestyle='--', label='True value')
    axes[i].set_title(f'Posterior for {["Intercept", "Coefficient 1", "Coefficient 2"][i]}')
    axes[i].legend()

plt.tight_layout()
plt.show()

In [ ]:
# Summarize posterior statistics
posterior_summary = pd.DataFrame({
    'Parameter': ['Intercept', 'Coefficient 1', 'Coefficient 2'],
    'True Value': true_beta,
    'Posterior Mean': np.mean(posterior_samples, axis=0),
    'Posterior Median': np.median(posterior_samples, axis=0),
    'Posterior Std': np.std(posterior_samples, axis=0),
    '5% Quantile': np.percentile(posterior_samples, 5, axis=0),
    '95% Quantile': np.percentile(posterior_samples, 95, axis=0)
})

posterior_summary

In [ ]:
# Plot decision boundary
plt.figure(figsize=(10, 6))

# Plot data points
plt.scatter(X[:, 0], X[:, 1], c=y, cmap='coolwarm', alpha=0.7)

# Create a grid of points
x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx1, xx2 = np.meshgrid(np.linspace(x1_min, x1_max, 100), np.linspace(x2_min, x2_max, 100))
grid = np.c_[xx1.ravel(), xx2.ravel()]

# Compute predicted probabilities for each point in the grid
grid_with_intercept = np.column_stack([np.ones(grid.shape[0]), grid])

# Use posterior mean for prediction
beta_mean = np.mean(posterior_samples, axis=0)
logits = grid_with_intercept @ beta_mean
probs = 1 / (1 + np.exp(-logits))

# Reshape probabilities to grid shape
probs = probs.reshape(xx1.shape)

# Plot decision boundary (p=0.5) and probability contours
plt.contour(xx1, xx2, probs, levels=[0.5], colors='k', linestyles='--')
plt.contourf(xx1, xx2, probs, alpha=0.3, cmap='coolwarm')

# Plot true decision boundary
# For logistic regression, the decision boundary is a line: beta[0] + beta[1]*x1 + beta[2]*x2 = 0
# Solving for x2: x2 = -(beta[0] + beta[1]*x1) / beta[2]
x1_line = np.array([x1_min, x1_max])
x2_line_true = -(true_beta[0] + true_beta[1] * x1_line) / true_beta[2]
plt.plot(x1_line, x2_line_true, 'r-', label='True boundary')

# Plot posterior mean decision boundary
x2_line_mean = -(beta_mean[0] + beta_mean[1] * x1_line) / beta_mean[2]
plt.plot(x1_line, x2_line_mean, 'g-', label='Posterior mean boundary')

plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Bayesian Logistic Regression: Decision Boundary')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Plot posterior uncertainty in decision boundary
plt.figure(figsize=(10, 6))

# Plot data points
plt.scatter(X[:, 0], X[:, 1], c=y, cmap='coolwarm', alpha=0.7)

# Plot a sample of decision boundaries from the posterior
n_samples_to_plot = 100
sample_indices = np.random.choice(len(posterior_samples), n_samples_to_plot, replace=False)

for idx in sample_indices:
    beta_sample = posterior_samples[idx]
    x2_line_sample = -(beta_sample[0] + beta_sample[1] * x1_line) / beta_sample[2]
    plt.plot(x1_line, x2_line_sample, 'b-', alpha=0.05)

# Plot true decision boundary
plt.plot(x1_line, x2_line_true, 'r-', linewidth=2, label='True boundary')

# Plot posterior mean decision boundary
plt.plot(x1_line, x2_line_mean, 'g-', linewidth=2, label='Posterior mean boundary')

plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Bayesian Logistic Regression: Posterior Uncertainty in Decision Boundary')
plt.legend()
plt.grid(True)
plt.show()

### 4.5 Comparison with Metropolis Algorithm

Let's compare the performance of HMC with the Metropolis algorithm for this logistic regression problem.

In [ ]:
# Define log probability function for Metropolis
def log_prob_logistic(beta):
    """Log probability for Bayesian logistic regression."""
    return -U_logistic(beta)

# Run Metropolis sampler
metropolis_samples, metropolis_acceptance_rate = metropolis_sampler(
    log_prob_logistic, initial_position, n_samples=5000, proposal_std=0.05, burn_in=1000, thin=2
)

print(f"Metropolis acceptance rate: {metropolis_acceptance_rate:.2f}")

In [ ]:
# Plot diagnostics for Metropolis
plot_diagnostics(metropolis_samples, parameter_names=['Intercept', 'Coefficient 1', 'Coefficient 2'])

In [ ]:
# Calculate effective sample size for each method
metropolis_ess = np.array([calculate_ess(metropolis_samples[:, i]) for i in range(3)])
hmc_ess = np.array([calculate_ess(posterior_samples[:, i]) for i in range(3)])

# Create comparison DataFrame
comparison_df = pd.DataFrame({
    'Parameter': ['Intercept', 'Coefficient 1', 'Coefficient 2'],
    'Metropolis ESS': metropolis_ess,
    'Metropolis Efficiency (ESS/N)': metropolis_ess / len(metropolis_samples),
    'HMC ESS': hmc_ess,
    'HMC Efficiency (ESS/N)': hmc_ess / len(posterior_samples),
    'Efficiency Ratio (HMC/Metropolis)': hmc_ess / metropolis_ess
})

comparison_df

In [ ]:
# Plot efficiency comparison
plt.figure(figsize=(10, 6))

x = np.arange(len(comparison_df))
width = 0.35

plt.bar(x - width/2, comparison_df['Metropolis Efficiency (ESS/N)'], width, label='Metropolis')
plt.bar(x + width/2, comparison_df['HMC Efficiency (ESS/N)'], width, label='HMC')

plt.xlabel('Parameter')
plt.ylabel('Sampling Efficiency (ESS/N)')
plt.title('Sampling Efficiency: Metropolis vs HMC for Logistic Regression')
plt.xticks(x, comparison_df['Parameter'])
plt.legend()
plt.grid(True, axis='y')
plt.show()

## 5. Limitations and Extensions of HMC

### 5.1 Limitations

Hamiltonian Monte Carlo, while powerful, has several limitations:

1. **Requires Gradient Information**: HMC requires the gradient of the log probability, which may not always be available or may be computationally expensive to compute.

2. **Tuning Parameters**: HMC has two main tuning parameters (step size and number of leapfrog steps) that need to be carefully chosen for optimal performance.

3. **Computational Cost**: Each HMC iteration requires multiple gradient evaluations (one for each leapfrog step), making it more computationally expensive than simpler methods like Metropolis or Gibbs sampling.

4. **Difficulty with Discrete Parameters**: HMC is designed for continuous parameters and cannot be directly applied to discrete parameters.

5. **Sensitivity to Ill-Conditioned Posteriors**: HMC can struggle with ill-conditioned posteriors where the scales of different parameters vary widely.

### 5.2 Extensions and Alternatives

Several extensions and alternatives address these limitations:

1. **No-U-Turn Sampler (NUTS)**: An adaptive version of HMC that automatically tunes the number of leapfrog steps, eliminating the need to manually specify this parameter.

2. **Riemannian HMC**: Uses a position-dependent mass matrix to better handle ill-conditioned posteriors.

3. **Dual Averaging**: A method for automatically tuning the step size to achieve a target acceptance rate.

4. **Hamiltonian Monte Carlo with Constraints**: Extensions of HMC that can handle constraints on the parameter space.

5. **Discontinuous HMC**: Variants of HMC that can handle discontinuities in the target distribution.

## 6. Conclusion

In this notebook, we've explored the Hamiltonian Monte Carlo algorithm, an advanced MCMC method that uses concepts from physics to efficiently sample from complex probability distributions. We've implemented it from scratch and applied it to sample from multivariate normal distributions and to perform Bayesian logistic regression.

Key takeaways:

1. HMC uses gradient information to propose states that are distant from the current state but still have a high probability of acceptance, reducing random walk behavior.

2. The performance of HMC depends on two key parameters: the step size and the number of leapfrog steps. These need to be carefully tuned for optimal performance.

3. HMC is particularly effective for high-dimensional problems with complex geometries, where simpler methods like Metropolis and Gibbs sampling can struggle.

4. The main drawback of HMC is its computational cost, as each iteration requires multiple gradient evaluations.

5. Extensions like NUTS address some of the limitations of basic HMC by automatically tuning the parameters.

In the next notebook, we'll explore how to implement these examples using PyMC, which provides efficient implementations of HMC and its extensions.